<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Cleaning_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import every dataset

In [18]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

Saving train_df_descr_analyse.json to train_df_descr_analyse.json


In [19]:
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

train_df_subclass = pd.read_json('trainset_subclass_instance.json', orient="records", lines=True)
train_df_images = pd.read_json('train_df_images.json', orient="records", lines=True)
train_df_dates = pd.read_json('train_df_dates.json', orient="records", lines=True)
train_df_lang = pd.read_json('train_df_lang.json', orient="records", lines=True)
train_df_country = pd.read_json('train_df_country.json', orient="records", lines=True)
train_df_descr = pd.read_json('train_df_descr_analyse.json', orient="records", lines=True)

# Merge all the datasets

In [20]:
train_df.columns.to_list()

['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']

In [22]:
# Get the dataset we first had
initial_df = train_df
initial_variables = train_df.columns.to_list()

# Get the datasets with the features we created
df_to_merge = [train_df_subclass, train_df_images, train_df_dates, train_df_lang, train_df_country, train_df_descr]

# Merge all of them, in a single dataframe
for df in df_to_merge:
  initial_df = pd.merge(left=initial_df, right=df, on=initial_variables, how='inner')

# Visualisation :
initial_df.head()

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image,date,nb_lang,main_country,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,descr_has_ethnic_group,descr_has_event
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,0,1,1,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1...",1949.0,126.0,Q298,2,0,1,0,1,0
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,0,1,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2...",1592.0,72.0,Q153136,4,0,1,0,1,0
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,0,1,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,...",165.0,83.0,Q1747689,1,0,1,0,1,0
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,0,1,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]...",1933.0,42.0,Q1041,2,0,1,0,1,0
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,0,1,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],...",1913.0,174.0,Q30,0,0,0,1,0,0


# Cleaning

## Labels

In [ ]:
initial_df['label'].describe()

,label
count,6244
unique,9
top,cultural exclusive
freq,2685


In [ ]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic', 'cultural', 'cult', nan, 'cultural agn',
       'cultural represent', 'cultural ex', 'cultural ag'], dtype=object)

In [ ]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cult,1
cultural,5
cultural ag,1
cultural agn,3
cultural agnostic,1862
cultural ex,1
cultural exclusive,2685
cultural represent,1
cultural representative,1685


We are suppose to have only 3 different labels

- Cultural Exclusive (C.E.)
- Cultural Agnostic (C.A.)
- Cultural Representative (C.R.)

Instead, we have seen that there are 9 different labels in the dataset, plus some NaN. So we have to clean the data, in order to get only 3 differents labels. Here is the solution we propose :

First step :

We distinguish the 3 labels we want (C.A., C.E., C.R.) and also labels that are close to this ('Cultural agn', 'cultural represent', 'cultural ex', 'cultural ag'). So we propose the following mapping :

- 'cultural agn' : C.A.
- 'cultural represent' : C.R.
- 'cultural ex' : C.E.
- 'cultural ag' : C.A.

Second step :

For the other labels ('cultural', 'cult', nan), we propose to apply the rule that was define to build the dataset (see https://huggingface.co/datasets/sapienzanlp/nlp2025_hw1_cultural_dataset) : Ask ChatGPT-o3 !

First step :

In [ ]:
to_correct = initial_df[initial_df["label"].isin(['cultural agn','cultural represent', 'cultural ex', 'cultural ag'])][['item', 'label']]
to_correct

,item,label
1545,http://www.wikidata.org/entity/Q86135347,cultural agn
3698,http://www.wikidata.org/entity/Q509900,cultural represent
4391,http://www.wikidata.org/entity/Q643677,cultural ex
4819,http://www.wikidata.org/entity/Q1711593,cultural agn
5731,http://www.wikidata.org/entity/Q25618,cultural ag
5968,http://www.wikidata.org/entity/Q30405,cultural agn


In [ ]:
mapping_correction = {
  'cultural agn' : 'cultural agnostic',
  'cultural represent' : 'cultural representative',
  'cultural ex' : 'cultural exclusive',
  'cultural ag' : 'cultural agnostic'
}

In [ ]:
to_correct['label'] = to_correct['label'].map(mapping_correction)

In [ ]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_correct.set_index('item', inplace=True)

initial_df.update(to_correct)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[1543:1547]

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image
1543,http://www.wikidata.org/entity/Q27503001,professional athlete,person who earns their living from sports,concept,sports,athlete,cultural agnostic,0,1,9,None
1544,http://www.wikidata.org/entity/Q107690317,competition climber,climber who competes in IFSC and Olympic climb...,concept,sports,athlete,cultural agnostic,0,3,9,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 2], ..."
1545,http://www.wikidata.org/entity/Q86135347,climber,person who practices climbing,concept,sports,athlete,cultural agnostic,2,1,9,None
1546,http://www.wikidata.org/entity/Q28971125,cheerleader,performer who leads crowd support at sports ev...,concept,sports,athlete,cultural representative,0,1,9,None


Second step :

Here are the items that ChatGPT will have to classify :

In [ ]:
to_classify = initial_df[(initial_df["label"].isin(["cultural", "cult"])) | (initial_df["label"].isna())][['item', 'name', 'description']]

to_classify

,item,name,description
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character
1066,http://www.wikidata.org/entity/Q30327019,post and lintel,building system where horizontal elements (bea...
1214,http://www.wikidata.org/entity/Q811361,architectural glass,building material typically used as transparen...
1651,http://www.wikidata.org/entity/Q12014207,trekking,backpacking or hiking
2556,http://www.wikidata.org/entity/Q257907,Ethiopian movement,religious movement in southern Africa
2831,http://www.wikidata.org/entity/Q1136336,Costa Book Awards,annual series of literary awards in five categ...
3236,http://www.wikidata.org/entity/Q67111,Franz Pfemfert,German journalist (1879-1954)
3820,http://www.wikidata.org/entity/Q1089672,The Elm-Chanted Forest,1986 animated film directed by Milan Blažeković
4853,http://www.wikidata.org/entity/Q85755629,Daniel Airlie,novel
5566,http://www.wikidata.org/entity/Q206912,extremophile,organisms specifically adapted to live and sur...


Here is the result for chatgpt :

In [ ]:
categories = {
    'http://www.wikidata.org/entity/Q957033': 'cultural representative',
    'http://www.wikidata.org/entity/Q30327019': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q811361': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q12014207': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q257907': 'cultural exclusive',
    'http://www.wikidata.org/entity/Q1136336': 'cultural representative',
    'http://www.wikidata.org/entity/Q67111': 'cultural representative',
    'http://www.wikidata.org/entity/Q1089672': 'cultural representative',
    'http://www.wikidata.org/entity/Q85755629': 'cultural representative',
    'http://www.wikidata.org/entity/Q206912': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q23228': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q3196604': 'cultural representative',
    'http://www.wikidata.org/entity/Q1940624': 'cultural representative'
}

to_classify['label'] = to_classify['item'].map(categories)

In [ ]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_classify.set_index('item', inplace=True)

initial_df.update(to_classify)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[856:860]

,item,name,description,type,category,subcategory,label,subclasses,instances_of,subclass_depth,image
856,http://www.wikidata.org/entity/Q929866,Santi Santamaria,Spanish chef (1957–2011),entity,food,cook,cultural exclusive,0,1,1,"[[[3, 3, 5], [3, 3, 5], [3, 3, 5], [3, 3, 5], ..."
857,http://www.wikidata.org/entity/Q899011,Ian Beale,fictional character from the soap opera EastEn...,entity,food,cook,cultural exclusive,0,2,1,"[[[81, 57, 38], [88, 57, 40], [94, 61, 40], [8..."
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character,entity,food,cook,cultural representative,0,4,1,None
859,http://www.wikidata.org/entity/Q935079,SpongeBob SquarePants,main character from the animated television sh...,entity,food,cook,cultural representative,0,4,1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


Check if everything is normal now :

In [ ]:
initial_df['label'].describe()

,label
count,6251
unique,3
top,cultural exclusive
freq,2687


In [ ]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic'], dtype=object)

In [ ]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cultural agnostic,1871
cultural exclusive,2687
cultural representative,1693


## Image